# AgentFleet: Multi-Agent Emergency Response System
## Agents for Good — A Comprehensive Demonstration

Welcome to the AgentFleet demonstration! This capstone project showcases a **multi-agent system designed to save lives and protect communities** through intelligent emergency response and incident management using Google's Agent Development Kit (ADK).

### 🌍 Why This Matters: Agents for Good

AgentFleet represents how AI agents can benefit entire communities by:
- **Reducing Response Time**: Automating incident triage and dispatch coordination
- **Improving Accuracy**: Verifying claims and assessing reliability across multiple sources
- **Saving Lives**: Prioritizing critical incidents for faster emergency response
- **Supporting First Responders**: Handling data processing so human operators can focus on action

This demo is a **proof-of-concept for what's possible**—imagine scaling this to coordinate emergency response across cities, regions, or entire nations.

### 🎯 Project Overview

AgentFleet is an advanced multi-agent system that processes emergency events through a sophisticated pipeline of specialized AI agents:

1. **Ingest Agent** - Normalizes raw event data from various sources (social media, sensors, emergency feeds)
2. **Verifier Agent** - Verifies claims and assesses source reliability to reduce false alarms
3. **Summarizer Agent** - Creates concise incident summaries with historical context for operators
4. **Triage Agent** - Classifies incident severity (LOW, MEDIUM, HIGH, CRITICAL) for prioritization
5. **Dispatcher Agent** - Generates actionable recommendations and communicates with response teams
6. **Dashboard Agent** - Creates markdown dashboards for human operators to visualize and manage emergency response operations

### 🏗️ Architecture

```mermaid
graph TD
    A[Event Sources] --> B[Ingest Agent]
    B --> C[Verifier Agent]
    C --> D[Summarizer Agent]
    D --> E[Triage Agent]
    E --> F[Dispatcher Agent]
    F --> G[Emergency Response Teams]
```

### 📋 Table of Contents

## 1. Setup and Configuration

This section initializes the AgentFleet system and configures dependencies for the multi-agent emergency response pipeline.

### 1.1 Project Root Configuration

First, we establish the project root directory by locating the `capstone` module. 
This ensures the notebook can access all capstone modules regardless of where it's executed from.




In [1]:
# %%writefile fleet-agent/agent.py

import sys
from pathlib import Path

# Walk up until we find "capstone" folder
current = Path.cwd()
for parent in [current] + list(current.parents):
    if (parent / "capstone" / "__init__.py").exists():
        sys.path.insert(0, str(parent))
        print("📁 Project root set to:", parent)
        break

import capstone
print("✅ capstone imported successfully!")

📁 Project root set to: /Users/ademola-ari/Projects/5-day-ai-agents-intensive
✅ capstone imported successfully!


### 1.2 Google API Authentication
Next, we authenticate with Google's API services. The system attempts to load credentials from environment variables, with a fallback to manual entry

In [2]:
import os
import dotenv

# Load environment variables from .env file if it exists
dotenv.load_dotenv()

# read from environment variable
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# if missing, prompt user to enter the API key
if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = input("Enter your Google API key: ")

if not GOOGLE_API_KEY:
    print("❌ Error: Google API key is required. Please set the 'GOOGLE_API_KEY' environment variable or enter it when prompted.")
else:
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Google API key obtained successfully.")


✅ Google API key obtained successfully.


### 1.3 ADK Components Import
We then import all necessary Agent Development Kit (ADK) modules and dependencies:

In [3]:
# %%writefile -a fleet-agent/agent.py

import json
import requests
import subprocess
import time
import uuid

from google.adk.agents import Agent, LlmAgent, SequentialAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)

from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner, Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Hide additional warnings in the notebook
import warnings

warnings.filterwarnings("ignore")

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### 1.4 Retry Configuration
Finally, we configure HTTP retry logic for resilient API communication:

In [4]:
# %%writefile -a fleet-agent/agent.py

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

print("✅ Retry configuration set.")

✅ Retry configuration set.


## 2. Ingest Agent Setup

The Ingest Agent is the first component in the AgentFleet pipeline. It connects to multiple event sources, normalizes incoming data, and prepares events for downstream agents.

### 2.1 Starting the Ingest Agent A2A Server

We launch the Ingest Agent as a background service using **Uvicorn**, an ASGI application server. This makes the agent accessible via HTTP as an Agent-to-Agent (A2A) service:

- **Service Type**: Python application running on localhost
- **Port**: 8001
- **Protocol**: HTTP REST with well-known agent card discovery
- **Module**: `capstone.agents.ingest_agent:app`

The notebook waits for the server to be ready before proceeding, polling the agent card endpoint every 5 seconds with a maximum of 20 attempts.

In [5]:
# %%writefile -a fleet-agent/agent.py

ingest_agent_code = '''
import os
from typing import Any, Dict
import dotenv

from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

from capstone.tools.stream_simulators import TwitterStreamSimulator, EmergencyFeedSimulator, SensorDataSimulator

def generate_single_event(source_type: str) -> Dict[str, Any]:
    """
    Generate a single event from the specified source type.
    
    Args:
        source_type: Type of stream source ("twitter", "emergency", "sensor")
        
    Returns:
        Dictionary containing the generated event data
        
    Raises:
        ValueError: If source_type is not supported
    """
    # Create simulator based on source type
    if source_type == "twitter":
        simulator = TwitterStreamSimulator()
    elif source_type == "emergency":
        simulator = EmergencyFeedSimulator()
    elif source_type == "sensor":
        simulator = SensorDataSimulator()
    else:
        return {
            "status": "error",
            "message": f"Unsupported source_type: {source_type}"
        }
    
    result = {
        "status": "success",
        "source_type": source_type,
        "event": simulator.generate_event()
    }
    return result

# Load environment variables from .env file if it exists
dotenv.load_dotenv()

# read from environment variable
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# if missing, prompt user to enter the API key
if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = input("Enter your Google API key: ")

if not GOOGLE_API_KEY:
    print("❌ Error: Google API key is required. Please set the 'GOOGLE_API_KEY' environment variable or enter it when prompted.")
else:
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Google API key obtained successfully.")


retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

INGEST_AGENT_INSTRUCTIONS = """You are the Ingest Agent in the AgentFleet incident response system.

Your responsibilities:
1. Call `generate_single_event` to generate a single event from a specified source.
2. The event output should be described in detail for further processing.

When processing events:
- Use the `generate_single_event` tool to generate events
- If the event is successfully generated, describe the generated event in detail 
- Generate unique event IDs
- Extract all relevant information from the event source and describe it in detail

User will supply the source to extract from e.g "twitter", "emergency_feed", or "sensor_data". Pass this to the `generate_single_event` tool to generate the event.
Then describe the generated event in detail in your output.
"""

# Create the Ingest Agent
ingest_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="ingest_agent",
    description="Ingest agent that connects to event sources and extract imformation for processing",
    instruction=INGEST_AGENT_INSTRUCTIONS,
    tools=[generate_single_event],
    output_key="raw_event_json",
)

# Create the A2A app
app = to_a2a(
    ingest_agent, port=8001
)
'''

with open("../agents/ingest_agent.py", "w") as f:
    f.write(ingest_agent_code)

print("✅ Ingest Agent code written to capstone/agents/ingest_agent.py")

✅ Ingest Agent code written to capstone/agents/ingest_agent.py


In [6]:
# %%writefile -a fleet-agent/agent.py


import subprocess, os
from pathlib import Path
import capstone

project_root = Path(capstone.__file__).resolve().parents[1]


# Setup to run in background
# run agent setup code in capstone/agents/ingest_agent.py for background execution

server_process = subprocess.Popen(
    [
        "uvicorn",
        "capstone.agents.ingest_agent:app",
        "--host",
        "localhost",
        "--port",
        "8001",
    ],
    cwd=project_root,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ},
)


print("🚀 Starting Ingest Agent A2A server in the background on port 8001.")
print("   Waiting for server to be ready...")

# Wait for the server to start
max_attempts = 20
for attempt in range(max_attempts):
    try:
        response = requests.get(
            "http://localhost:8001/.well-known/agent-card.json", timeout=1
        )
        if response.status_code == 200:
            print("✅ Ingest Agent A2A server is ready.")
            print("   Server URL: http://localhost:8001")
            print("   Agent Card URL: http://localhost:8001/.well-known/agent-card.json")
            break
    except requests.exceptions.RequestException:
        time.sleep(5)
        print(".", end="", flush=True)

else:
    print("\n⚠️  Server may not be ready yet. Check manually if needed.")

# Store the process so it can be terminated later if needed
globals()["ingest_agent_server_process"] = server_process

🚀 Starting Ingest Agent A2A server in the background on port 8001.
   Waiting for server to be ready...
.✅ Ingest Agent A2A server is ready.
   Server URL: http://localhost:8001
   Agent Card URL: http://localhost:8001/.well-known/agent-card.json


### 2.2 View the Auto-Generated Agent Card

Once the Ingest Agent server is running, we retrieve and display its **agent card**—a standardized JSON document that describes the agent's capabilities, exposed skills, and communication interface.

The agent card includes:
- **Name**: Unique identifier for the agent
- **Description**: Purpose and role in the incident response system
- **URL**: HTTP endpoint for agent-to-agent communication
- **Skills**: List of capabilities and tools the agent exposes to other agents

This discovery mechanism enables other agents to dynamically learn what the Ingest Agent can do without hardcoding dependencies.


In [7]:
# %%writefile -a fleet-agent/agent.py

try:
    response = requests.get(
        "http://localhost:8001/.well-known/agent-card.json", timeout=5
    )
    if response.status_code == 200:
        agent_card = response.json()
        print("✅ Ingest Agent A2A server is reachable.")
        print(json.dumps(agent_card, indent=2))

        print("\n Key details:")
        print(f" - Name: {agent_card.get('name')}")
        print(f" - Description: {agent_card.get('description')}")
        print(f" - URL: {agent_card.get('url')}")
        print(f" - Skills: {len(agent_card.get('skills', []))} capabilities exposed")
    else:
        print(
            f"❌ Failed to reach Ingest Agent A2A server. Status code: {response.status_code}"
        )
except requests.exceptions.RequestException as e:
    print(f"❌ Error reaching Ingest Agent A2A server: {e}")

✅ Ingest Agent A2A server is reachable.
{
  "capabilities": {},
  "defaultInputModes": [
    "text/plain"
  ],
  "defaultOutputModes": [
    "text/plain"
  ],
  "description": "Ingest agent that connects to event sources and extract imformation for processing",
  "name": "ingest_agent",
  "preferredTransport": "JSONRPC",
  "protocolVersion": "0.3.0",
  "skills": [
    {
      "description": "Ingest agent that connects to event sources and extract imformation for processing I am the Ingest Agent in the AgentFleet incident response system.\n\nmy responsibilities:\n1. Call `generate_single_event` to generate a single event from a specified source.\n2. The event output should be described in detail for further processing.\n\nWhen processing events:\n- Use the `generate_single_event` tool to generate events\n- If the event is successfully generated, describe the generated event in detail \n- Generate unique event IDs\n- Extract all relevant information from the event source and describe it 

### 2.3 Create Client-Side Proxy to Interact with the Ingest Agent A2A App

We instantiate a **RemoteA2aAgent** proxy that acts as a local client for the remote Ingest Agent service. This proxy:

- Connects to the agent card endpoint on port 8001
- Translates local requests into A2A protocol messages
- Handles serialization/deserialization of data
- Enables seamless communication between the notebook and the Ingest Agent service

The proxy is then added as a sub-agent to the root orchestrating agent, allowing the entire pipeline to work together as a coordinated multi-agent system.

In [8]:
# %%writefile -a fleet-agent/agent.py

remote_ingest_agent = RemoteA2aAgent(
    name="remote_ingest_agent",
    description="Client-side proxy to interact with the Ingest Agent A2A app",
    agent_card=f"http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("✅ RemoteA2aAgent proxy created for Ingest Agent A2A app.")
print(f"   Connected to: http://localhost:8001")
print(f"   Agent card: http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}")
print("   Ready to send requests to Ingest Agent.")

✅ RemoteA2aAgent proxy created for Ingest Agent A2A app.
   Connected to: http://localhost:8001
   Agent card: http://localhost:8001/.well-known/agent-card.json
   Ready to send requests to Ingest Agent.


![Ingest Agent](../demo/ingest_agent.png)

### Test A2A Interaction with Ingest Agent

In [9]:
# Create a test user agent to interact with the Ingest Agent A2A app
remote_ingest_agent_test = RemoteA2aAgent(
    name="remote_ingest_agent_test",
    description="Client-side proxy to interact with the Ingest Agent A2A app - test instance",
    agent_card=f"http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("✅ RemoteA2aAgent proxy created for Ingest Agent A2A app.")
print(f"   Connected to: http://localhost:8001")
print(f"   Agent card: http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}")
print("   Ready to send requests to Ingest Agent.")

emergency_response_tester_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="emergency_response_tester_agent",
    description="Test agent to interact with the Ingest Agent A2A app",
    instruction="""You are a test agent that interacts with the Ingest Agent A2A app to simulate emergency event ingestion.
    Your tasks:
    1. Send requests to the Ingest Agent to ingest events from a simulated stream.
    2. Receive and display the normalized events returned by the Ingest Agent.
    3. Validate that the events are correctly normalized and contain required fields: location, time, event type.
    4. Report any issues or discrepancies found in the normalized events.
    """,
    sub_agents=[remote_ingest_agent_test],
)

print("✅ Emergency Response Tester Agent created.")
print("   Model: gemini-2.5-flash-lite")
print("   Ready to simulate event ingestion via Ingest Agent.")

✅ RemoteA2aAgent proxy created for Ingest Agent A2A app.
   Connected to: http://localhost:8001
   Agent card: http://localhost:8001/.well-known/agent-card.json
   Ready to send requests to Ingest Agent.
✅ Emergency Response Tester Agent created.
   Model: gemini-2.5-flash-lite
   Ready to simulate event ingestion via Ingest Agent.


In [10]:
async def test_ingest_agent_a2a_interaction(query: str):
    """
    Test the A2A communication between the Emergency Response Tester Agent and the Ingest Agent.

    Steps:
    1. The tester agent sends a request to the Ingest Agent to ingest to get events from a simulated stream.
    2. The Ingest Agent processes the request, connects to the stream, normalizes events, and returns the results.
    3. The tester agent receives and displays the normalized events

    Args:
        query (str): The request/query to send to the Ingest Agent.
    """

    session_service = InMemorySessionService()

    # Session identifiers
    app_name = "emergency_response_app"
    user_id = "tester_user"

    session_id = f"test_session_{uuid.uuid4().hex[:8]}"

    session = await session_service.create_session(
        app_name=app_name, user_id=user_id, session_id=session_id
    )

    # Create runner for the tester agent
    runner = Runner(
        agent=emergency_response_tester_agent,
        app_name=app_name,
        session_service=session_service,
    )

    # Create the user message
    test_content = types.Content(parts=[types.Part(text=query)])

    # Display query
    print(f"\n🧪 Emergency Response Tester: {query}")
    print(f".    Ingest Agent response:")
    print("..................................................")

    # Run the agent asynchronously (handles streaming responses and A2A communication)
    async for event in runner.run_async(
        user_id=user_id,
        session_id=session_id,
        new_message=test_content,
    ):
        # Print final response only (skip intermediate events)
        if event.is_final_response() and event.content:
            for part in event.content.parts:
                if hasattr(part, "text"):
                    print(part.text)

    print("." * 80)
    print("✅ Test completed.")

In [11]:
# Example test interaction
print("\n=== Testing A2A Interaction with Ingest Agent ===")
await test_ingest_agent_a2a_interaction(
    "Ingest latest events from twitter"
)


=== Testing A2A Interaction with Ingest Agent ===

🧪 Emergency Response Tester: Ingest latest events from twitter
.    Ingest Agent response:
..................................................


The event generated from twitter is about a power outage in Oak Avenue. The power restoration efforts are ongoing and may take hours. The event has a tweet ID, user information, and timestamp. It also includes hashtags like #alert. The incident type is logged as power_outage with associated likes and retweets.
................................................................................
✅ Test completed.


## 3. Verifier Agent Setup

The Verifier Agent is the second component in the AgentFleet pipeline. It receives normalized events from the Ingest Agent and validates claims by fact-checking them against reliable sources, then scores the overall reliability of each event.

### 3.1 Import Verifier Tools

We begin by importing the specialized tools that the Verifier Agent uses to perform its verification tasks:


In [12]:
# %%writefile -a fleet-agent/agent.py

# setup verifier agent to receive events from the ingest agent and verify claims
from capstone.agents.verifier_agent import (
    extract_claims_tool,
    verify_claim_tool,
    score_reliability_tool
)

VERIFIER_AGENT_INSTRUCTION = """
You are the Verifier Agent in the AgentFleet incident response system.

Your responsibilities:
1. Receive event summary from the Ingest Agent
2. Extract verifiable claims from event content
3. Fact-check claims using search tools (when available)
4. Score source reliability based on verification results

When processing events:
- Use the `extract_claims_tool` to identify factual statements that can be verified
- Use the `verify_claim_tool` to check each claim (with search results if available)
- Use the `score_reliability_tool` to calculate overall reliability score (0.0 to 1.0)
- Flag events with reliability score below 0.3 as unverified
- Explain your reasoning for each claim verification and reliability score in the output

Be thorough in verification but efficient in processing.
Always include session_id for tracking incident lifecycle.
"""

verifier_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="verifier_agent",
    description="Verifier agent in the AgentFleet incident response system",
    instruction=VERIFIER_AGENT_INSTRUCTION,
    tools=[extract_claims_tool, verify_claim_tool, score_reliability_tool],
    output_key="verified_claims",
)

print("✅ Verifier Agent Successfully Setup")
print("   Model: gemini-2.5-flash-lite")
print("   Tools: extract_claims_tool(), verify_claim_tool(), score_reliability_tool()")
print("   Ready to verify claims from ingested events.")

✅ Verifier Agent Successfully Setup
   Model: gemini-2.5-flash-lite
   Tools: extract_claims_tool(), verify_claim_tool(), score_reliability_tool()
   Ready to verify claims from ingested events.


![Verifier Agent](../demo/verifier_agent.png)

## 4. Summarizer Agent Setup

The Summarizer Agent is the third component in the AgentFleet pipeline. It receives verified events from the Verifier Agent and transforms them into concise, actionable incident briefs for human operators.

### 4.1 Import Summarizer Tools

We begin by importing the specialized tool that the Summarizer Agent uses to extract and structure key information:

The Summarizer Agent performs the following functions:

- **Receives verified events** with reliability scores from the Verifier Agent
- **Generates concise incident briefs** (maximum 200 words) suitable for rapid operator review
- **Extracts key facts** including location, time, affected entities, and impact assessment
- **Structures output** with organized information for downstream agents (Triage and Dispatcher)
- **Explains reasoning** about which facts were included and why they're important

The summarizer uses the `extract_key_facts_tool` to identify and organize critical information from verified claims, ensuring that operators receive clear, focused summaries that enable faster decision-making in emergency response scenarios.

**Key Capabilities:**
- Concise summarization (max 200 words per incident brief)
- Structured fact extraction (location, time, entities, impact)
- Pattern identification and similarity scoring
- Seamless handoff to Triage Agent for severity classification

This agent bridges the gap between verification and incident prioritization, ensuring that critical information is neither lost nor buried in unnecessary detail.

In [13]:
# %%writefile -a fleet-agent/agent.py


from capstone.agents.summarizer_agent import extract_key_facts_tool

SUMMARIZER_AGENT_INSTRUCTIONS = """
You are the Summarizer Agent in the AgentFleet incident response system.

Your responsibilities:
1. Receive verified events from the Verifier Agent
2. Generate concise incident briefs (maximum 200 words)
3. Extract key facts: location, time, affected entities, impact
4. Output incident briefs with structured key facts

When processing events:
- Generate concise summaries (max 200 words)
- Use the `extract_key_facts_tool` to identify structured information
- Include similarity scores and patterns in the incident brief
- Explain your reasoning in the output, including which facts were included and why

Be concise but comprehensive. Focus on actionable information.
Ensure summaries are clear and suitable for human operators.

Verifier Agent Event: {verified_claims?}
"""

summarizer_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="summarizer_agent",
    description="Summarizer agent in the AgentFleet incident response system",
    instruction=SUMMARIZER_AGENT_INSTRUCTIONS,
    tools=[extract_key_facts_tool],
    output_key="incident_brief"
)

print("✅ Summarizer Agent Successfully Setup")
print("   Model: gemini-2.5-flash-lite")
print("   Tool: extract_key_facts_tool()")
print("   Ready to summarize verified events into incident briefs.")

✅ Summarizer Agent Successfully Setup
   Model: gemini-2.5-flash-lite
   Tool: extract_key_facts_tool()
   Ready to summarize verified events into incident briefs.


![Summarizer Agent](../demo/summarizer_agent.png)

## 5. Triage Agent Setup

The Triage Agent is the fourth component in the AgentFleet pipeline. It receives incident briefs from the Summarizer Agent and classifies them by severity level to prioritize emergency response.

### 5.1 Import Triage Tools

We begin by importing the specialized tools that the Triage Agent uses to classify incidents and manage the response queue:

The Triage Agent performs the following functions:

- **Receives incident briefs** from the Summarizer Agent with key facts and summaries
- **Analyzes incident content** to determine the appropriate severity classification
- **Classifies incidents** into four levels: LOW, MEDIUM, HIGH, or CRITICAL
- **Calculates priority scores** (0.0 to 1.0) for incident ordering and dispatch sequencing
- **Creates job queue entries** for HIGH and CRITICAL incidents to alert response teams
- **Forwards triaged incidents** to the Dispatcher Agent for action generation

**Severity Classification Guidelines:**
- **CRITICAL**: Immediate threat to life, major infrastructure failure, widespread impact (>1000 affected)
- **HIGH**: Significant threat, infrastructure damage, regional impact (100–1000 affected)
- **MEDIUM**: Moderate threat, localized damage, limited impact (10–100 affected)
- **LOW**: Minor threat, minimal damage, very limited impact (<10 affected)

**Key Capabilities:**
- Intelligent severity analysis based on multiple factors (threat level, scope, casualties)
- Priority scoring for efficient incident sequencing
- Job queue creation for automated escalation
- Reasoning transparency for operator review and audit trails

This agent serves as the critical decision point in the pipeline, ensuring that limited emergency resources are deployed to the most urgent incidents first, ultimately saving lives through intelligent prioritization.

In [14]:
# %%writefile -a fleet-agent/agent.py

from capstone.agents.triage_agent import (
    classify_severity_tool,
    create_job_tool,
    )

TRIAGE_AGENT_INSTRUCTION = """You are the Triage Agent in the AgentFleet incident response system.

Your responsibilities:
1. Receive incident briefs from the Summarizer Agent
2. Analyze incident content to determine severity level
3. Classify incidents as LOW, MEDIUM, HIGH, or CRITICAL based on:
   - Threat to life or safety
   - Infrastructure impact
   - Geographic scope
   - Number of people affected
   - Urgency of response needed
4. Calculate priority scores (0.0 to 1.0) for incident ordering
5. Create job queue entries for HIGH and CRITICAL incidents
6. Forward triaged incidents to the Dispatcher Agent

Severity Classification Guidelines:
- CRITICAL: Immediate threat to life, major infrastructure failure, widespread impact (>1000 affected)
- HIGH: Significant threat, infrastructure damage, regional impact (100-1000 affected)
- MEDIUM: Moderate threat, localized damage, limited impact (10-100 affected)
- LOW: Minor threat, minimal damage, very limited impact (<10 affected)

When processing incidents:
- Use the `classify_severity_tool` to analyze and classify incidents
- Use the `create_job_tool` to create job entries for HIGH and CRITICAL incidents
- Always provide clear reasoning for severity assignments
- Consider reliability scores when making classifications
- Explain your reasoning in the output, including which factors influenced the severity classification

Be decisive and consistent in your classifications.
Prioritize human safety and infrastructure integrity.

Summarizer Agent Event: {incident_brief?}
"""

triage_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="triage_agent",
    description="Classify incident severity as LOW, MEDIUM, HIGH, or CRITICAL with priority score",
    instruction=TRIAGE_AGENT_INSTRUCTION,
    tools=[classify_severity_tool, create_job_tool],
    output_key="triage_result"
)

print("✅ Triage Agent created to classify incident severity and create job entries.")
print("   Ready to receive incident briefs and perform triage.")
print()

✅ Triage Agent created to classify incident severity and create job entries.
   Ready to receive incident briefs and perform triage.



![Triage Agent](../demo/triage_agent.png)

## 6. Dispatcher Agent Setup

The Dispatcher Agent is the fifth and final component in the AgentFleet pipeline. It receives triaged incidents from the Triage Agent and converts them into concrete action plans, communication templates, and persistent records for emergency response teams.

### 6.1 Import Dispatcher Tools

We begin by importing the specialized tools that the Dispatcher Agent uses to generate actions, create communications, and persist incident data:

The Dispatcher Agent performs the following functions:

- **Receives triaged incidents** from the Triage Agent with severity classifications and priority scores
- **Generates recommended actions** with specific steps, responsible parties, and realistic timelines
- **Creates communication templates** for HIGH and CRITICAL severity incidents to alert response teams
- **Persists incident data** to the database for historical tracking and audit trails
- **Notifies the Operator Dashboard** to make incidents immediately visible to human operators
- **Updates job status** to mark incidents as processed and ready for response

**Action Generation Guidelines:**
- Provide specific, actionable recommendations tailored to severity level
- Assign clear responsible parties (e.g., Fire Department, Police, Emergency Medical Services)
- Include realistic response timelines based on incident severity and scope
- Prioritize human safety and critical infrastructure protection above all else
- Scale response scope appropriately (e.g., local response for LOW, regional coordination for CRITICAL)

**Communication Template Guidelines:**
- Use professional, clear, and concise language suitable for emergency personnel
- Include all critical incident information (ID, severity level, location, brief summary)
- List the top 3–5 priority actions in order of importance
- Provide contact information placeholders for inter-agency coordination
- Set clear expectations for next status updates and response milestones

**Key Capabilities:**
- Multi-format action generation (structured recommendations for automated systems)
- Template-based communication for rapid team briefing
- Complete incident data persistence for regulatory compliance and post-incident analysis
- Real-time dashboard updates for situational awareness
- Seamless integration with existing emergency response infrastructure

This agent serves as the bridge between intelligent incident analysis and human-driven emergency response, ensuring that operators have everything needed to act decisively and effectively. By automating routine communication and documentation tasks, the Dispatcher Agent frees response teams to focus on saving lives.

In [15]:
# %%writefile -a fleet-agent/agent.py

from capstone.agents.dispatcher_agent import (
    generate_actions_tool,
    create_communication_template_tool,
    persist_incident_tool,
    notify_dashboard_tool,
)

DISPATCHER_AGENT_INSTRUCTION = """You are the Dispatcher Agent in the AgentFleet incident response system.

Your responsibilities:
1. Receive triaged incidents from the Triage Agent
2. Generate recommended actions with specific steps, responsible parties, and timelines
3. Create communication templates for HIGH and CRITICAL severity incidents
4. Persist complete incident data to the database
5. Update job status to COMPLETED
6. Notify the Operator Dashboard of new incidents

Action Generation Guidelines:
- Provide specific, actionable recommendations
- Assign clear responsible parties
- Include realistic timelines based on severity
- Prioritize human safety and infrastructure protection
- Scale response appropriately to severity level

Communication Template Guidelines:
- Use professional, clear language
- Include all critical information (ID, severity, location, summary)
- List top priority actions
- Provide contact information placeholders
- Set expectations for next updates

When processing incidents:
- Use the `generate_actions_tool` to create recommended actions
- Use the `create_communication_template_tool` for HIGH/CRITICAL incidents
- Use the `persist_incident_tool` to save to database
- Use the `notify_dashboard_tool` to make incident available to operators
- Always ensure complete data persistence
- Confirm job status is updated to COMPLETED

Be thorough and professional in all communications.
Ensure operators have all information needed to respond effectively.

Triage Agent Event: {triage_result?}
"""

dispatcher_agent = LlmAgent(
    name="dispatcher_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Dispatcher agent in the AgentFleet incident response system",
    instruction=DISPATCHER_AGENT_INSTRUCTION,
    tools=[
        generate_actions_tool,
        create_communication_template_tool,
        persist_incident_tool,
        notify_dashboard_tool,
    ],
    output_key="dispatch_result",
)

print("✅ Dispatcher Agent created to generate actions, persist incidents, and notify dashboard.")
print("   Ready to receive triaged incidents and perform dispatching.")

✅ Dispatcher Agent created to generate actions, persist incidents, and notify dashboard.
   Ready to receive triaged incidents and perform dispatching.


![Dispatcher Agent](../demo/dispatcher_agent.png)

### Dashboard Agent Setup
The Dashboard Agent is an additional component in the AgentFleet pipeline. It receives dispatched incidents from the Dispatcher Agent and creates markdown dashboards for human operators to visualize and manage emergency response operations.

The Dashboard Agent performs the following functions:
- **Receives dispatched incidents** from the Dispatcher Agent with recommended actions and communication templates
- **Generates markdown dashboards** that summarize incident details, severity levels, recommended actions, and status updates
- **Organizes dashboards** for easy navigation and quick reference by human operators


In [16]:
# %%writefile -a fleet-agent/agent.py

DASHBOARD_AGENT_INSTRUCTION = """You are the Dashboard Agent in the AgentFleet incident response system.

Your responsibilities:
1. Access the INCIDENT_CACHE from the Dispatcher Agent to get current incident data
2. Create and maintain an incident dashboard with all current incidents
3. Display incident information in a clear, organized markdown table format
4. Include all relevant fields: incident ID, severity, status, location, priority score, actions, and timestamps
5. Update the dashboard with new incidents as they are dispatched

Dashboard Requirements:
- Create a markdown table with the following columns:
  - Incident ID
  - Severity Level (LOW/MEDIUM/HIGH/CRITICAL)
  - Status (DISPATCHED, IN_PROGRESS, RESOLVED, etc.)
  - Priority Score (0.0-1.0)
  - Location (if available)
  - Summary (brief description)
  - Actions Required (count of recommended actions)
  - Dispatched At (timestamp)
- Sort incidents by severity (CRITICAL → HIGH → MEDIUM → LOW) and then by priority score (highest first)
- Include all dispatched incidents in the dashboard
- Format timestamps in a readable format (YYYY-MM-DD HH:MM UTC)
- Keep summaries concise (max 50 characters)
- Display the dashboard as markdown output that can be rendered in notebooks
- Highlight critical and high-priority incidents with detailed action items
- Include communication templates for critical/high incidents

When processing incidents:
- Use the `create_dashboard_markdown_tool` to generate the dashboard from cache
- The tool automatically accesses the INCIDENT_CACHE from the Dispatcher Agent
- Focus on presenting information clearly for human operator review
- Ensure critical incidents are prominently displayed with action items

Be thorough in presenting all incident information clearly.
Focus on readability and actionable information for operators.

The dashboard will automatically reflect the current state of all dispatched incidents.
"""

from capstone.agents.dashboard_agent import create_dashboard_markdown_tool

dashboard_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="dashboard_agent",
    description="Dashboard agent to create and maintain incident response dashboard",
    instruction=DASHBOARD_AGENT_INSTRUCTION,
    tools=[create_dashboard_markdown_tool],
    output_key="dashboard_output"
)

print("✅ Dashboard Agent created to create and maintain incident response dashboard.")
print("   Model: gemini-2.5-flash-lite")
print("   Tool: create_dashboard_markdown_tool()")
print("   Ready to create markdown dashboards from dispatched incidents.")


✅ Dashboard Agent created to create and maintain incident response dashboard.
   Model: gemini-2.5-flash-lite
   Tool: create_dashboard_markdown_tool()
   Ready to create markdown dashboards from dispatched incidents.


![Dashboard Agent](../demo/dashboard_agent.png)

In [17]:
# Root agent to coordinate ingest and verifier agents

root_agent = SequentialAgent(
    name="root_agent",
    description="Root agent to coordinate Ingest, Verifier, Summarizer, Triage, and Dispatcher agents in the AgentFleet incident response system",
    sub_agents=[remote_ingest_agent, verifier_agent, summarizer_agent, triage_agent, dispatcher_agent],
)
print("✅ Root Agent created to coordinate Ingest, Verifier, Summarizer, Triage, and Dispatcher agents.")
print("   Ready to manage end-to-end event ingestion, verification, summarization, triage, and dispatch.")
print()
print("===" * 20)

# Root agent runner
root_agent_runner = InMemoryRunner(
    agent=root_agent,
    app_name="root_agent_app",
)
print("✅ Root Agent Runner created.")
print("   Ready to run end-to-end event ingestion, verification, summarization, triage, and dispatch workflows.")
print()
print("===" * 20)



✅ Root Agent created to coordinate Ingest, Verifier, Summarizer, Triage, and Dispatcher agents.
   Ready to manage end-to-end event ingestion, verification, summarization, triage, and dispatch.

✅ Root Agent Runner created.
   Ready to run end-to-end event ingestion, verification, summarization, triage, and dispatch workflows.



In [18]:
# agent in action
print("\n=== Running Root Agent to Ingest, Verify, Summarize, Triage, and Dispatch Events ===")
response = await root_agent_runner.run_debug("Ingest latest events from twitter")

INFO:httpx:HTTP Request: GET http://localhost:8001/.well-known/agent-card.json "HTTP/1.1 200 OK"
INFO:a2a.client.card_resolver:Successfully fetched agent card data from http://localhost:8001/.well-known/agent-card.json: {'capabilities': {}, 'defaultInputModes': ['text/plain'], 'defaultOutputModes': ['text/plain'], 'description': 'Ingest agent that connects to event sources and extract imformation for processing', 'name': 'ingest_agent', 'preferredTransport': 'JSONRPC', 'protocolVersion': '0.3.0', 'skills': [{'description': 'Ingest agent that connects to event sources and extract imformation for processing I am the Ingest Agent in the AgentFleet incident response system.\n\nmy responsibilities:\n1. Call `generate_single_event` to generate a single event from a specified source.\n2. The event output should be described in detail for further processing.\n\nWhen processing events:\n- Use the `generate_single_event` tool to generate events\n- If the event is successfully generated, describe


=== Running Root Agent to Ingest, Verify, Summarize, Triage, and Dispatch Events ===

 ### Created new session: debug_session_id

User > Ingest latest events from twitter


INFO:httpx:HTTP Request: POST http://localhost:8001 "HTTP/1.1 200 OK"
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False


remote_ingest_agent > The event was generated successfully from the twitter source.

Event ID: 0b6a237c-d0a0-4e08-9cc5-44e59649603d
Event Source: twitter
Timestamp: 2025-11-29T21:59:09.438792

Event Details:
A structure fire is occurring on Main Street with heavy smoke reported. The tweet was posted by user_7123 and has 832 likes and 273 retweets. The incident type is classified as a fire.


INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:capstone.agents.verifier_agent:Extracted 2 claims from event
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:capstone.agents.verifier_agent:Verified claim with confidence 0.27
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:capstone.agents.verifier_agent:Verified claim with confidence 0.31
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:capstone.agents.verifier_

verifier_agent > The event from twitter has been analyzed. The following claims were extracted:
1. "A structure fire is occurring on Main Street with heavy smoke reported"
2. "The tweet was posted by user_7123 and has 832 likes and 273 retweets"

Verification results:
- Claim 1 was not verified. There were no corroborating sources found, and the confidence score was low (0.27).
- Claim 2 was verified. Although no external sources were used for verification in this instance, the claim pertains to tweet-specific metadata, which is considered inherently verifiable by the platform. The confidence score was 0.31.

Overall reliability:
The overall reliability score for this event is 0.37. This score is based on one verified claim out of two extracted claims (50% verification rate) and an average confidence score of 0.29. While the event contains some verifiable information, the lack of external corroboration for the primary assertion (structure fire) leads to a moderate reliability score. Fu

INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:capstone.agents.summarizer_agent:Extracted 1 key facts
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False


summarizer_agent > A structure fire was reported on Main Street with heavy smoke. The original tweet was posted by user_7123 and received 832 likes and 273 retweets.

**Key Facts:**
*   **Location:** Main Street
*   **Time:** 2025-11-29T21:59:09.438792
*   **Affected Entity:** A structure
*   **Impact:** Heavy smoke reported; classified as a fire.
*   **Source:** Twitter (user_7123)
*   **Engagement:** 832 likes, 273 retweets

**Reasoning:**
The primary claim regarding the structure fire was not externally verified (confidence: 0.27), though the tweet's metadata (likes, retweets) was considered inherently verifiable (confidence: 0.31). The overall reliability score is 0.37. Due to the unverified nature of the core incident report, further investigation into the fire claim is recommended. The key facts extracted focus on the reported details and the source of the information, while also noting the low confidence in the fire report itself. The incident type was included as 'fire' based o

INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:capstone.agents.triage_agent:Classified incident as LOW (priority: 0.10)
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:capstone.agents.dispatcher_agent:Generated 3 recommended actions for incident 0b6a237c-d0a0-4e08-9cc5-44e59649603d
ERROR:capstone.agents.dispatcher_agent:Error persisting incident: no such table: incidents
INFO:capstone.agents.dispatcher_agent:Added incident 0b6a237c-d0a0-4e08-9cc5-44e59649603d to dashboard cache
INFO:google_adk.google.adk.models.google_llm:Sending out request, mod

dispatcher_agent > The incident has been logged and the dashboard has been updated. Recommended actions are:
- Log incident for record keeping and trend analysis (Responsible: Operations Center, Timeline: Within 24 hours)
- Review and update standard operating procedures if needed (Responsible: Planning Section, Timeline: Within 1 week)
- Coordinate with local authorities in Main Street (Responsible: Regional Coordinator, Timeline: As appropriate for severity level)

The incident data could not be persisted to the database due to a table not being found. Please ensure the database is set up correctly.


In [19]:
# if fleet-agent folder does not exist, create it
if not os.path.exists("fleet-agent"):
    !adk create fleet-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY

In [20]:
# Run web server for the fleet-agent
RUN_ADK_WEB_SERVER = False
if RUN_ADK_WEB_SERVER:
    !adk web --log_level DEBUG

![Demo](../demo/agent-fleet.gif)


## 7. Demo Summary

This demonstration showcased the AgentFleet multi-agent emergency response system, highlighting how specialized AI agents can collaborate to ingest, verify, summarize, triage, dispatch, and visualize emergency incidents. By automating critical tasks and providing actionable insights, AgentFleet aims to enhance community safety and support first responders in saving lives.
